In [2]:
print("hi")

hi


## dependencies
- %pip install langchain-community faiss-cpu
- %pip install -qU langchain_community pypdf
- %pip install sentence-transformers
- %pip install openai



# Load PDFs

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("496.pdf")
document = loader.load()

# Chunking Text

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(document)

# Embedding generation

In [6]:
from langchain.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
embeddings = embedding_model.embed_documents([text.page_content for text in texts])

/var/folders/0h/q4qdtr9j7g791902612bqpgh0000gn/T/ipykernel_76636/4188097725.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/Users/victor/Documents/Documents - Victor’s MacBook Air (2)/ai_project/RAG_AI_venv/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Vector Storage

In [7]:
from langchain.vectorstores import FAISS

vectorstore = FAISS.from_documents(texts, embedding_model)

# Retreival and response generation

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
import os
from dotenv import load_dotenv

load_dotenv()

# setting up the llm using the api key from goettingen
llm = ChatOpenAI(
    openai_api_key=os.getenv("API_KEY"),
    openai_api_base="https://chat-ai.academiccloud.de/v1",
    model_name="meta-llama-3.1-8b-instruct",
    temperature=0.7
)

In [10]:
from langchain.chains import RetrievalQA

retriever = vectorstore.as_retriever()

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

In [11]:
query = "What are the main topics in the text?"
result = qa_chain({"query": query})

print("Answer:\n", result["result"])
print("\nSources:\n", result["source_documents"])

/var/folders/0h/q4qdtr9j7g791902612bqpgh0000gn/T/ipykernel_76636/3321119944.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": query})


Answer:
 The main topics in the text are:

1. Discursive politics of environmental reproductive justice
2. The importance of centering community voices in policy-making
3. The role of language and discourse in shaping social and political meaning
4. The application of a critical, interpretive, and discursive approach to the study of politics and public administration.

Overall, the text appears to be discussing the ways in which language and discourse can be used to shape policy and maintain power, and the importance of amplifying marginalized voices in policy-making.

Sources:
 [Document(id='62909f6d-f2d2-4b7b-9da7-93bb50438333', metadata={'producer': 'Adobe PDF Library 17.0; modified using iText 4.2.0 by 1T3XT', 'creator': 'Adobe InDesign 19.5 (Windows)', 'creationdate': '2024-12-10T12:38:52+05:30', 'moddate': '2025-03-31T05:29:32-07:00', 'trapped': '/False', 'subject': 'Politics & Policy 2024.52:1246-1265', 'wps-proclevel': '3', 'wps-journaldoi': '10.1111/(ISSN)1747-1346', 'title': 